In [196]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np


In [91]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/user/Downloads/chromedriver 2", options=options)
    driver.set_window_size(1120, 1000)
    
    #Locaiton urls need to be manualy selected because glassdoor website can be buggy
    #"San Francisco
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    #No location
    #url= 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox'
    #Seattle
    #url='https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox&locId=1150505&locT=C&locName=Seattle%2C%20WA%20(US)'
    #New York
    #url='https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox&locId=1132348&locT=C&locName=New%20York%2C%20NY%20(US)'
    #Boston
    #url='https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox&locId=1154532&locT=C&locName=Boston%2C%20MA%20(US)'
    #Austin
    #url='https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox&locId=1139761&locT=C&locName=Austin%2C%20TX%20(US)'
    #remote
    url='https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword.replace(" ","%20") + '&suggestCount=0&suggestChosen=false&clickSource=searchBox&locId=11047&locT=S&locName=Remote%20(Work%20From%20Home)%2C%20US'
    driver.get(url)
    jobs = []

    
    

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_css_selector('[id="onetrust-accept-btn-handler"]').click()  #clicking to cookies
        except:
            pass
    
        time.sleep(.5)
        
        try:
            
            driver.find_element_by_css_selector('[id="PanesWrap"] article ul[data-test="jlGrid"] li:first-child').click()  #clicking to the X.
        except:
            pass

        time.sleep(.5)
        
        try:
            driver.find_element_by_css_selector('[class="SVGInline-svg modal_closeIcon-svg"]').click()
        except:
            pass
        

        #Going through each job in this page
        job_buttons = driver.find_elements_by_css_selector('[id="PanesWrap"] article ul[data-test="jlGrid"] li')  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            temp_element = job_button
            # create action chain object
            action = ActionChains(driver)

            # perform the operation
            action.move_to_element(temp_element).perform()
            
            temp_element.click()  #You might
            
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_css_selector('div[data-brandviews^="BRAND:n=jsearch-hero-header"] > div:last-child > div[class^="css"] > div:nth-child(1)').text.split("\n")[0]
                    location = driver.find_element_by_css_selector('div[data-brandviews^="BRAND:n=jsearch-hero-header"] > div:last-child > div[class^="css"] > div:nth-child(3)').text
                    job_title = driver.find_element_by_css_selector('div[data-brandviews^="BRAND:n=jsearch-hero-header"] > div:last-child > div[class^="css"] > div:nth-child(2)').text
                    job_description = driver.find_element_by_css_selector('[class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_css_selector('[class="salaryTab tabSection p-std"] > div[class^="css"] > div[class^="css"] > div[class^="css"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_css_selector('[data-test="detailRating"]').text
                
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_css_selector('[data-test="scrollable-tabs"] [data-test="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    #headquarters = driver.find_element_by_css_selector('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    headquarters = -1
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(1)  span:nth-child(2)').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(2)  span:nth-child(2)').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(3)  span:nth-child(2)').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(4)  span:nth-child(2)').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(5)  span:nth-child(2)').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_css_selector('[id="EmpBasicInfo"] div[class="d-flex flex-wrap"] > div:nth-child(6)  span:nth-child(2)').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    #competitors = driver.find_element_by_css_selector('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                    competitors = -1
                except NoSuchElementException:
                    competitors = -1
                
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
        #Clicking on the "next page" button
        try:
            temp_selector = '[data-test="pagination-next"]'
            temp_element = driver.find_element_by_css_selector(temp_selector)
            # create action chain object
            action = ActionChains(driver)

            # perform the operation
            action.move_to_element(temp_element).perform()
            driver.find_element_by_css_selector(temp_selector).click()
        except:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break;
    driver.quit()
    return pd.DataFrame(jobs)

In [92]:
#df_sanfran = get_jobs("data scientist", 600, True)
#df_national = get_jobs("data scientist", 600, True)
#df_Seattle = get_jobs("data scientist", 600, True)
#df_NY = get_jobs("data scientist", 600, True)
#df_Bos = get_jobs("data scientist", 600, True)
#df_Austin = get_jobs("data scientist", 600, True)
df_remote = get_jobs("data scientist", 600, True)




Progress: 0/600
Job Title: Entry Level Data Scientist
Salary Estimate: -1
Job Description: Toyota Connected's Mobility team is responsible for building and managing our connected vehicle platforms, supporting product research using vehicle sensor data and creating new and exciting data services for Toyota customers that makes driving safer, more convenient, and fun. We're looking for an Entry Level Data Scientist interested in using data science and statistical techniques to create state-of-the-art solutions for non-trivial, and arguably, unsolved problems. If you are results driven, 
Rating: -1
Company Name: Toyota Connected North America
Location: Plano, TX
Headquarters: -1
Size: 51 to 200 Employees
Founded: 2016
Type of Ownership: Company - Private
Industry: IT Services
Sector: Information Technology
Revenue: Unknown / Non-Applicable
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 1/600
Job Title: Data Scientist - remote
Salary Estimate: -1
Job Descrip

Headquarters: -1
Size: 10000+ Employees
Founded: 1931
Type of Ownership: Company - Public
Industry: Insurance Agencies & Brokerages
Sector: Insurance
Revenue: $10+ billion (USD)
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 10/600
Job Title: Data Scientist Contract-W2
Salary Estimate: $13.06 /hr (est.)
Job Description: Collecting data through means such as analyzing business results or by setting up and managing new studies
Transferring data into a new format to make it more appropriate for analysis
Creating new, experimental frameworks to collect data
Building tools to automate data collection
Searching through large data sets for usable information
Creating reports and presentations for business uses
Correlating similar data to find actionable results
Job Types: Full-time, Contract
Rating: -1
Company Name: Snowrelic
Location: Remote
Headquarters: -1
Size: Unknown
Founded: Company - Public
Type of Ownership: Unknown / Non-Applicable
Industry: -1
Sector

Headquarters: -1
Size: 1001 to 5000 Employees
Founded: 2008
Type of Ownership: Company - Private
Industry: Health, Beauty, & Fitness
Sector: Consumer Services
Revenue: Unknown / Non-Applicable
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 20/600
Job Title: Data Scientist
Salary Estimate: $11.87 /hr (est.)
Job Description: Skill Sets:
Bachelor’s, Master’s or PhD degree in Statistics/Economics/Math/Computer Science or related field
Experience in SQL, R, Python to implement statistical models, machine learning, and analysis (Recommenders, Prediction, Classification, Clustering, etc.) in big data environment
Experience on large scale computing systems like COSMOS, Hadoop, MapReduce and/or similar systems preferred
Experience with programming skills, e.g. Java, C# is a plus
Exceptional written and verbal communication
Rating: 4.1
Company Name: Rangam Consultants Inc.
Location: Bellevue, WA
Headquarters: -1
Size: 501 to 1000 Employees
Founded: 1995
Type of Ow

Headquarters: -1
Size: 51 to 200 Employees
Founded: Company - Private
Type of Ownership: Research & Development
Industry: Business Services
Sector: Less than $1 million (USD)
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 30/600
Job Title: Staff Data Scientist - Remote USA
Salary Estimate: $150,000 /yr (est.)
Job Description: Company Description

FireEye is the intelligence-led security company. Working as a seamless, scalable extension of customer security operations, FireEye offers a single platform that blends innovative security technologies, nation-state grade threat intelligence, and world-renowned Mandiant® consulting. With this approach, FireEye eliminates the complexity and burden of cyber security for organizations struggling to prepare for, prevent, and respond to cyber attacks. Learn more about FireEye's 
Rating: 3.8
Company Name: FireEye, Inc.
Location: New York, NY
Headquarters: -1
Size: 1001 to 5000 Employees
Founded: 2004
Type

Job Title: Machine Learning Engineer - Remote
Salary Estimate: -1
Job Description: Company Description

CapTech is a team of master builders, creators, and problem solvers who help clients grow efficient, successful businesses. We unite diverse skills and perspectives to transform how data, systems, and ingenuity enable each client to advance what’s possible in a changing world.
As perceptive partners, our U.S-based consultants find inspiration in the unknown and enjoy getting our hands dirty solving our clients’ myriad of challenges. Across industries and business goals, we f
Rating: 3.9
Company Name: CapTech Consulting
Location: Raleigh, NC
Headquarters: -1
Size: 1001 to 5000 Employees
Founded: 1997
Type of Ownership: Company - Private
Industry: IT Services
Sector: Information Technology
Revenue: $100 to $500 million (USD)
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 40/600
Job Title: Data Scientist
Salary Estimate: $102,500 /yr (est.)
Job Descriptio

...

Job Title: Senior Data Scientist
Salary Estimate: -1
Job Description: We’re looking for problem solvers, innovators, and dreamers who are searching for anything but business as usual. Like us, you’re a high performer who’s an expert at your craft, constantly challenging the status quo. You value inclusivity and want to join a culture that empowers you to show up as your authentic self. You know that success hinges on commitment, that our differences make us stronger, and that the finish line is always sweeter when the whole team crosses together.
Alteryx is search
Rating: 3.6
Company Name: Alteryx, Inc.
Location: Detroit, MI
Headquarters: -1
Size: 1001 to 5000 Employees
Founded: 1997
Type of Ownership: Company - Public
Industry: Enterprise Software & Network Solutions
Sector: Information Technology
Revenue: $100 to $500 million (USD)
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 553/600
Job Title: Machine Learning Engineer
Salary Estimate: -1
Job Descri

Job Title: Data Scientist Contract-W2
Salary Estimate: $13.06 /hr (est.)
Job Description: Collecting data through means such as analyzing business results or by setting up and managing new studies
Transferring data into a new format to make it more appropriate for analysis
Creating new, experimental frameworks to collect data
Building tools to automate data collection
Searching through large data sets for usable information
Creating reports and presentations for business uses
Correlating similar data to find actionable results
Job Types: Full-time, Contract
Rating: -1
Company Name: Snowrelic
Location: Remote
Headquarters: -1
Size: Unknown
Founded: Company - Public
Type of Ownership: Unknown / Non-Applicable
Industry: -1
Sector: -1
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 563/600
Job Title: Machine Learning Engineer
Salary Estimate: $100,000 /yr (est.)
Job Description: Scanslated, Inc. is a digital health startup. We use AI software to a

Headquarters: -1
Size: 201 to 500 Employees
Founded: Company - Private
Type of Ownership: Research & Development
Industry: Business Services
Sector: $25 to $50 million (USD)
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 572/600
Job Title: Clinical Data Scientist
Salary Estimate: -1
Job Description: Job Summary:
Data mining of genomics data and clinical record. The work can be remote.
Essential Duties and Responsibilities:
*
DUTIES AND RESPONSIBILITIES: *
Conduct research on large datasets
Discover novel patterns with statistically significance
Provide models to guidance clinical product development
Job Skills:
Rating: 2.6
Company Name: Baylor Genetics
Location: Houston, TX
Headquarters: -1
Size: 201 to 500 Employees
Founded: 2015
Type of Ownership: Company - Private
Industry: Health Care Services & Hospitals
Sector: Health Care
Revenue: $50 to $100 million (USD)
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 5

Headquarters: -1
Size: 51 to 200 Employees
Founded: 2014
Type of Ownership: Company - Private
Industry: Enterprise Software & Network Solutions
Sector: Information Technology
Revenue: Unknown / Non-Applicable
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 582/600
Job Title: Data Scientist
Salary Estimate: $19.00 /hr (est.)
Job Description: Location: Remote
Start Date: ASAP
Duration: 6 months
Additional Details:
40 hours of work per week
Remote or on site in Malvern (PA), Charlotte (NC), or Dallas (TX) [whatever candidate is
comfortable with]
6 months to start with possible extensions
Senior AWS Data Scientist Consultant x4
Rating: -1
Company Name: PSY LLC
Location: Remote
Headquarters: -1
Size: -1
Founded: -1
Type of Ownership: -1
Industry: -1
Sector: -1
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 583/600
Job Title: 0412WFH-04 Data Scientist
Salary Estimate: -1
Job Description: Clearance Level: Secret

US Ci

Headquarters: -1
Size: 1 to 50 Employees
Founded: 2018
Type of Ownership: Company - Private
Industry: Video Games
Sector: Media
Revenue: Unknown / Non-Applicable
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 592/600
Job Title: Senior Data Scientist
Salary Estimate: -1
Job Description: Senior Data Scientist
IT, InfoSec, Cyber Risk & Business Operations | San Francisco, CA or Seattle, WA
If this position is eligible for remote employment in the US, the employee can work remotely in all but the following states: Alaska, Hawaii, Iowa, Maine, Mississippi, North Dakota, South Dakota, Vermont, West Virginia and Wyoming.

Our agreement with employees
DocuSign is committed to building trust and making the world more agreeable for our employees, customers and the communities in which w
Rating: 4.4
Company Name: DocuSign
Location: San Francisco, CA
Headquarters: -1
Size: 5001 to 10000 Employees
Founded: 2003
Type of Ownership: Company - Public
Industry: Computer H

In [101]:
#no location
df_national
#san francisco
df_sanfran
#Seattle
df_Seattle
#New York
df_NY
#Austin
df_Austin
#Remote
df_Remote
#Boston
df_Bos


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,"As a Data Scientist, you will work with a team...",4.2,Recorded Future,"Boston, MA",-1,501 to 1000 Employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD),-1
1,Junior Data Scientist,-1,Pinata Scienes is looking for an up and coming...,-1,PIÑATA SCIENCES,"Boston, MA",-1,-1,-1,-1,-1,-1,-1,-1
2,Staff Data Scientist - Remote USA,"$150,000 /yr (est.)",Company Description\n\nFireEye is the intellig...,3.8,"FireEye, Inc.","Boston, MA",-1,1001 to 5000 Employees,2004,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
3,Data Scientist,-1,Pison is creating the next generation of neura...,2.0,Pison,"Boston, MA",-1,1 to 50 Employees,Company - Public,$1 to $5 million (USD),-1,-1,-1,-1
4,Data Scientist,-1,"Company Description\n\nPowerAdvocate, a Verisk...",4.5,PowerAdvocate,"Boston, MA",-1,51 to 200 Employees,1999,Company - Public,Energy,"Oil, Gas, Energy & Utilities",Unknown / Non-Applicable,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Data Scientist,-1,Overview:\n\nAnalysis Group is one of the larg...,4.0,"Analysis Group, Inc.","Boston, MA",-1,1001 to 5000 Employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
596,Data Scientist II,-1,Master’s degree (or Bachelors degree or 5+ yea...,3.8,Amazon.com Services LLC,"North Reading, MA",-1,10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD),-1
597,Data Scientist,-1,Company Summary: Korro Bio is a privately held...,-1,Korro Bio,"Cambridge, MA",-1,1 to 50 Employees,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,-1,-1
598,Senior Data Scientist,-1,Join our industry leading Data Science team at...,3.2,RUE GILT GROUPE,"Boston, MA",-1,501 to 1000 Employees,Company - Private,"Department, Clothing, & Shoe Stores",Retail,Unknown / Non-Applicable,-1,-1


In [102]:
df_Remote['Location']='Remote'
df_national['Location']='National'
df_Bos['Location']='Boston'
df_sanfran['Location']='San Francisco'
df_NY['Location']='NYC'
df_Seattle['Location']='Seattle'
df_Austin['Location']='Austin'


In [108]:
df_All=pd.concat([df_Remote,df_Austin,df_Bos,df_national,df_NY,df_sanfran,df_Seattle])


In [201]:
#find all salaries in the data, convert them to numeric
df_Salary=df_All[df_All['Salary Estimate']!=-1]
Salary=df_Salary['Salary Estimate'].apply(lambda x: x.split('/')[0])
Salary=Salary.apply(lambda x:x.replace("$",""))
Salary=Salary.str.replace(',','')
Salary=pd.to_numeric(Salary)
Salary
#487 observations


8      165000.00
9      154800.00
10         13.06
13         19.00
14         10.68
         ...    
534    289200.00
554    150000.00
564    289200.00
584    150000.00
594    289200.00
Name: Salary Estimate, Length: 487, dtype: float64

In [203]:
#Add cleaned hourly estimator (convert hourly into yearly) and create hourly indicator 
df_Salary['Salary']=Salary
df_Salary['Salary']
df_Salary['Salary'] = np.where(
    df_Salary['Salary'] < 2000,                
    df_Salary['Salary'] * 2000,   
    df_Salary['Salary']                     
)
df_Salary["Hourly"]=df_Salary['Salary Estimate'].apply(lambda x: 1 if '/hr' in x.lower() else 0)


<ipython-input-203-155c204ffdc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Salary['Salary']=Salary
<ipython-input-203-155c204ffdc8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Salary['Salary'] = np.where(


In [211]:
df_Salary.to_csv('/Users/user/Documents/Salary.csv')



In [206]:
df_Salary


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Salary,Hourly
8,Data Scientist,"$165,000 /yr (est.)",Data Scientist ($160-170 yr) Must have profess...,-1,"IdentifyUp, Inc.",Remote,-1,Unknown,Company - Public,Unknown / Non-Applicable,-1,-1,-1,-1,165000.0,0
9,Avail - Staff Data Scientist (Remote Home Base...,"$154,800 /yr (est.)",Avail is a car sharing service to help people ...,3.5,Allstate,Remote,-1,10000+ Employees,1931,Company - Public,Insurance Agencies & Brokerages,Insurance,$10+ billion (USD),-1,154800.0,0
10,Data Scientist Contract-W2,$13.06 /hr (est.),Collecting data through means such as analyzin...,-1,Snowrelic,Remote,-1,Unknown,Company - Public,Unknown / Non-Applicable,-1,-1,-1,-1,26120.0,1
13,Data Scientist,$19.00 /hr (est.),Location: Remote\nStart Date: ASAP\nDuration: ...,-1,PSY LLC,Remote,-1,-1,-1,-1,-1,-1,-1,-1,38000.0,1
14,Data Science/Data Scientist,$10.68 /hr (est.),Job title: Data Scientist\nLocation : Remote\n...,4.1,Celebal Technologies,Remote,-1,201 to 500 Employees,2016,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1,21360.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Senior Data Scientist - Deep Learning/Computer...,"$289,200 /yr (est.)",Senior Data Scientist - Deep Learning/Computer...,3.7,Nordstrom,Seattle,-1,10000+ Employees,1901,Company - Public,"Department, Clothing, & Shoe Stores",Retail,$10+ billion (USD),-1,289200.0,0
554,Data Scientist,"$150,000 /yr (est.)",We are adding a Data Scientist to our growing ...,4.3,American Capital Group,Seattle,-1,51 to 200 Employees,1986,Company - Private,Real Estate,Real Estate,Unknown / Non-Applicable,-1,150000.0,0
564,Senior Data Scientist - Deep Learning/Computer...,"$289,200 /yr (est.)",Senior Data Scientist - Deep Learning/Computer...,3.7,Nordstrom,Seattle,-1,10000+ Employees,1901,Company - Public,"Department, Clothing, & Shoe Stores",Retail,$10+ billion (USD),-1,289200.0,0
584,Data Scientist,"$150,000 /yr (est.)",We are adding a Data Scientist to our growing ...,4.3,American Capital Group,Seattle,-1,51 to 200 Employees,1986,Company - Private,Real Estate,Real Estate,Unknown / Non-Applicable,-1,150000.0,0
